# Entrenamiento de ResNet-18 con todos los conjuntos de MedMNIST

Este cuaderno prepara y entrena un modelo ResNet-18 capaz de clasificar de forma conjunta todos los conjuntos de MedMNIST. El flujo reutiliza el módulo `scripts/entrenar_resnet_general.py`

## Flujo de trabajo
- Importar utilidades y fijar una configuración reproducible
- Listar los datasets disponibles y definir la configuración de entrenamiento
- Preparar los *DataLoaders* combinando todos los conjuntos
- Construir el modelo ResNet-18 y los objetos de optimización
- Entrenar el modelo con seguimiento de métricas por época
- Evaluar en el conjunto de prueba y generar un reporte reutilizable

In [30]:
# Importaciones y configuración básica
import json
from pathlib import Path
from types import SimpleNamespace

import pandas as pd
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights


from medmnist import INFO

from scripts.Train_EN2 import (
    crear_cargadores,
    construir_modelo,
    recorrer_entrenamiento,
    evaluar_modelo,
    guardar_reporte,
    construir_scheduler,
    nn,
    Adam,
)

# Fijamos semillas para reproducibilidad básica
TORCH_SEED = 42
torch.manual_seed(TORCH_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(TORCH_SEED)

print(f"Versión de PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

Versión de PyTorch: 2.8.0+cpu
CUDA disponible: False


In [31]:
# Listado de datasets 2D de MedMNIST
# Filtramos los que no tienen '3d' en el nombre de su clase de Python
todos_los_datasets = sorted(
    nombre
    for nombre, info in INFO.items()
    if '3d' not in nombre.lower() and '3d' not in info.get('python_class', '').lower()
)
print(f"Total de datasets 2D: {len(todos_los_datasets)}")
todos_los_datasets

Total de datasets 2D: 12


['bloodmnist',
 'breastmnist',
 'chestmnist',
 'dermamnist',
 'octmnist',
 'organamnist',
 'organcmnist',
 'organsmnist',
 'pathmnist',
 'pneumoniamnist',
 'retinamnist',
 'tissuemnist']

In [32]:
# Configuración centralizada; ajusta hiperparámetros aquí
config = SimpleNamespace(
    datasets=todos_los_datasets,
    epocas=5,  # Incrementa si quieres un entrenamiento más largo
    tamano_lote=256,
    tasa_aprendizaje=1e-3,
    decaimiento_peso=1e-4,
    tamano_imagen=28,
    usar_aumentos=True,
    descargar=True,
    sin_preentrenar=False,
    dispositivo="cuda" if torch.cuda.is_available() else "cpu",
    salida="resultados/reporte_efficientnet_todos.json",
    trabajadores=4,
    scheduler="plateau",
    factor_scheduler=0.5,
    paciencia_scheduler=3,
)
config

namespace(datasets=['bloodmnist',
                    'breastmnist',
                    'chestmnist',
                    'dermamnist',
                    'octmnist',
                    'organamnist',
                    'organcmnist',
                    'organsmnist',
                    'pathmnist',
                    'pneumoniamnist',
                    'retinamnist',
                    'tissuemnist'],
          epocas=5,
          tamano_lote=256,
          tasa_aprendizaje=0.001,
          decaimiento_peso=0.0001,
          tamano_imagen=28,
          usar_aumentos=True,
          descargar=True,
          sin_preentrenar=False,
          dispositivo='cpu',
          salida='resultados/reporte_efficientnet_todos.json',
          trabajadores=4,
          scheduler='plateau',
          factor_scheduler=0.5,
          paciencia_scheduler=3)

## Preparar *DataLoaders* combinando todos los conjuntos

In [33]:
(
    loader_entrenamiento,
    loader_validacion,
    loader_prueba,
    info_datasets,
) = crear_cargadores(
    nombres_dataset=config.datasets,
    tamano_lote=config.tamano_lote,
    descarga=config.descargar,
    tamano_imagen=config.tamano_imagen,
    usar_aumentos=config.usar_aumentos,
    trabajadores=config.trabajadores,
)

resumen_datos = {
    "train": len(loader_entrenamiento.dataset),
    "test": len(loader_prueba.dataset),
}
if loader_validacion is not None:
    resumen_datos["val"] = len(loader_validacion.dataset)

print("Resumen de muestras por partición:", resumen_datos)

total_clases = sum(info.clases for info in info_datasets.values())
print("Total de clases combinadas:", total_clases)

info_datasets

Resumen de muestras por partición: {'train': 518175, 'test': 119320, 'val': 70467}
Total de clases combinadas: 92


{'bloodmnist': RegistroDataset(nombre='bloodmnist', indice_inicial=0, clases=8),
 'breastmnist': RegistroDataset(nombre='breastmnist', indice_inicial=8, clases=2),
 'chestmnist': RegistroDataset(nombre='chestmnist', indice_inicial=10, clases=14),
 'dermamnist': RegistroDataset(nombre='dermamnist', indice_inicial=24, clases=7),
 'octmnist': RegistroDataset(nombre='octmnist', indice_inicial=31, clases=4),
 'organamnist': RegistroDataset(nombre='organamnist', indice_inicial=35, clases=11),
 'organcmnist': RegistroDataset(nombre='organcmnist', indice_inicial=46, clases=11),
 'organsmnist': RegistroDataset(nombre='organsmnist', indice_inicial=57, clases=11),
 'pathmnist': RegistroDataset(nombre='pathmnist', indice_inicial=68, clases=9),
 'pneumoniamnist': RegistroDataset(nombre='pneumoniamnist', indice_inicial=77, clases=2),
 'retinamnist': RegistroDataset(nombre='retinamnist', indice_inicial=79, clases=5),
 'tissuemnist': RegistroDataset(nombre='tissuemnist', indice_inicial=84, clases=8)}

In [25]:
import inspect
from scripts.Train_EN2 import construir_modelo
print(inspect.getsource(construir_modelo))


def construir_modelo(total_clases: int, usar_pesos_imagenet: bool) -> nn.Module:
    """Inicializa la Efficient-BO ajustando la capa de salida al número total de clases."""
    if usar_pesos_imagenet:
        pesos = EfficientNet_B0_Weights.DEFAULT
        modelo = efficientnet_b0(weights=pesos)
    else:
        modelo = efficientnet_b0(weights=None)
    #caracteristicas = modelo.fc.in_features
    #modelo.fc = nn.Linear(caracteristicas, total_clases)
    #return modelo
    # EfficientNet-B0 tiene la cabeza en classifier[1]
    in_features = modelo.classifier[1].in_features
    modelo.classifier[1] = nn.Linear(in_features, total_clases)
    return modelo



In [26]:
import importlib
import scripts.Train_EN2
importlib.reload(scripts.Train_EN2)


<module 'scripts.Train_EN2' from 'e:\\ModeloGeneral-MedMNIST\\scripts\\Train_EN2.py'>

In [27]:
!grep -R "ResNet" -n e:/ModeloGeneral-MedMNIST/scripts/


"grep" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Construir modelo, criterio de pérdida y optimizador

In [34]:
dispositivo = torch.device(config.dispositivo)
print(f"Usando dispositivo: {dispositivo}")

modelo = construir_modelo(
    total_clases=total_clases,
    usar_pesos_imagenet=not config.sin_preentrenar,
)
modelo = modelo.to(dispositivo)

criterio = nn.CrossEntropyLoss()
optimizador = Adam(
    modelo.parameters(),
    lr=config.tasa_aprendizaje,
    weight_decay=config.decaimiento_peso,
)
scheduler = construir_scheduler(
    optimizador=optimizador,
    tipo_scheduler=config.scheduler,
    factor=config.factor_scheduler,
    paciencia=config.paciencia_scheduler,
)

modelo

Usando dispositivo: cpu
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\estef/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:02<00:00, 8.29MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

## Entrenar el modelo

In [35]:
historial_entrenamiento, mejor_estado = recorrer_entrenamiento(
    modelo=modelo,
    cargador_entrenamiento=loader_entrenamiento,
    cargador_validacion=loader_validacion,
    criterio=criterio,
    optimizador=optimizador,
    scheduler=scheduler,
    epocas=config.epocas,
    dispositivo=dispositivo,
)

# Si hubo mejora en validación, recargamos el mejor estado
if mejor_estado:
    modelo.load_state_dict(mejor_estado)

historial_entrenamiento[:3]

c:\Users\estef\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[Epoca 001] Perdida ent: 0.9508 | Exactitud ent: 0.6881 | Perdida val: 0.8010 | Exactitud val: 0.7310
[Epoca 002] Perdida ent: 0.7942 | Exactitud ent: 0.7338 | Perdida val: 0.7762 | Exactitud val: 0.7396
[Epoca 003] Perdida ent: 0.7656 | Exactitud ent: 0.7434 | Perdida val: 0.7345 | Exactitud val: 0.7548
[Epoca 004] Perdida ent: 0.7446 | Exactitud ent: 0.7493 | Perdida val: 0.7340 | Exactitud val: 0.7560
[Epoca 005] Perdida ent: 0.7289 | Exactitud ent: 0.7549 | Perdida val: 0.7049 | Exactitud val: 0.7615


[{'epoca': 1,
  'perdida_entrenamiento': 0.9508099685430975,
  'exactitud_entrenamiento': 0.6881458966565349,
  'perdida_validacion': 0.8009700593017657,
  'exactitud_validacion': 0.7309520768586715},
 {'epoca': 2,
  'perdida_entrenamiento': 0.794237363373554,
  'exactitud_entrenamiento': 0.7338119361219665,
  'perdida_validacion': 0.7762133454732437,
  'exactitud_validacion': 0.739551846963827},
 {'epoca': 3,
  'perdida_entrenamiento': 0.7655880914974668,
  'exactitud_entrenamiento': 0.7433897814444926,
  'perdida_validacion': 0.7344641606144933,
  'exactitud_validacion': 0.7547930236848454}]

## Historial de métricas por época

In [36]:
df_historial = pd.DataFrame(historial_entrenamiento)
df_historial

,epoca,perdida_entrenamiento,exactitud_entrenamiento,perdida_validacion,exactitud_validacion
0,1,0.950810,0.688146,0.800970,0.730952
1,2,0.794237,0.733812,0.776213,0.739552
2,3,0.765588,0.743390,0.734464,0.754793
3,4,0.744585,0.749334,0.733999,0.756028
4,5,0.728866,0.754926,0.704937,0.761505


## Evaluación en el conjunto de prueba

In [37]:
perdida_test, exactitud_test, metricas_por_dataset = evaluar_modelo(
    modelo,
    loader_prueba,
    criterio,
    dispositivo,
)

print(f"Exactitud global en test: {exactitud_test:.4f}")
print(f"Pérdida global en test: {perdida_test:.4f}")

filas = []
for nombre, datos in metricas_por_dataset.items():
    filas.append(
        {
            "dataset": nombre,
            "exactitud": datos.get("exactitud", 0.0),
            "ejemplos": int(datos.get("ejemplos", 0)),
            "offset": info_datasets[nombre].indice_inicial,
            "clases": info_datasets[nombre].clases,
        }
    )

df_metricas = pd.DataFrame(filas).sort_values("exactitud", ascending=False)
df_metricas

c:\Users\estef\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Exactitud global en test: 0.6936
Pérdida global en test: 0.9389


,dataset,exactitud,ejemplos,offset,clases
0,bloodmnist,0.907045,3421,0,8
5,organamnist,0.870795,17778,35,11
9,pneumoniamnist,0.863782,624,77,2
8,pathmnist,0.850000,7180,68,9
1,breastmnist,0.698718,156,8,2
6,organcmnist,0.661879,8216,46,11
3,dermamnist,0.647880,2005,24,7
11,tissuemnist,0.639425,47280,84,8
7,organsmnist,0.635777,8827,57,11
2,chestmnist,0.627914,22433,10,14


## Guardar reporte y revisar el resultado

In [38]:
metricas_reporte = {
    "perdida": perdida_test,
    "exactitud": exactitud_test,
    "detalle_por_dataset": df_metricas.set_index("dataset")[['exactitud', 'ejemplos']].to_dict(orient="index"),
}

ruta_reporte = Path(config.salida)

guardar_reporte(
    ruta_salida=ruta_reporte,
    argumentos=config,
    historial=historial_entrenamiento,
    metricas_prueba=metricas_reporte,
    informacion_datasets=info_datasets,
)

print(f"Reporte guardado en: {ruta_reporte.resolve()}")

with ruta_reporte.open("r", encoding="utf-8") as archivo:
    resumen = json.load(archivo)
resumen.keys()

Reporte guardado en: E:\ModeloGeneral-MedMNIST\resultados\reporte_efficientnet_todos.json


dict_keys(['fecha', 'configuracion', 'historial_epocas', 'metricas_prueba', 'datasets'])